<a href="https://colab.research.google.com/github/ehpub/KFQ_DL_2023-/blob/main/2023_kfq_%EB%94%A5%EB%9F%AC%EB%8B%9D_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 교재: https://www.gilbut.co.kr/book/view?bookcode=BN003496

In [1]:
import string

In [2]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
src = "aidfadf#$5C*4jc*#Jkei!"
"".join(char for char in src if char not in string.punctuation)

'aidfadf5C4jcJkei'

In [18]:
class Vectorizer:
  def stardardize(self,text):
    text = text.lower()
    return "".join(char for char in text if char not in string.punctuation)
  def tokenize(self,text):
    return text.split()
  def make_vocabulary(self,dataset):
    self.vocabulary = {"":0,
                       "[UNK]":1}
    for text in dataset:
      text = self.stardardize(text)
      tokens =self.tokenize(text)
      for token in tokens:
        if token not in self.vocabulary:
          self.vocabulary[token] = len(self.vocabulary)
      self.inverse_vocabulary = dict(
          (v,k) for k,v in self.vocabulary.items()
      )
  def encode(self,text):
    text = self.stardardize(text)
    tokens =self.tokenize(text)
    return [self.vocabulary.get(token,1)  for token in tokens ]
  def decode(self,int_sequence):
    return " ".join(self.inverse_vocabulary.get(i,"[UNK]") for i in int_sequence)



In [20]:
vect = Vectorizer()
data = ["I am a boy.",
 "You are a girl.",
 "show me the money."]
vect.make_vocabulary(data)
st = vect.stardardize("Hello, I am a gir. You are a mimi.")
tks = vect.tokenize(st)
for token in tks:
  print(vect.vocabulary.get(token,1))
wi = [1,4,5,6,9,10,8,7,5,89]
re =vect.decode(wi)
print(re)

1
2
3
4
1
6
7
4
1
[UNK] a boy you show me girl are boy [UNK]


In [22]:
from tensorflow import keras
from keras import layers

In [23]:
tv = layers.TextVectorization(output_mode='int')

In [25]:
tv.adapt(data) #앞에서 정의한 Vectorizer 클래스의 makevocabulary와 같은 역할

In [26]:
tv.get_vocabulary()

['',
 '[UNK]',
 'a',
 'you',
 'the',
 'show',
 'money',
 'me',
 'i',
 'girl',
 'boy',
 'are',
 'am']

In [27]:
tv("Hello, I am a gir. You are a mimi.") #encode를 수행

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([ 1,  8, 12,  2,  1,  3, 11,  2,  1])>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  49.0M      0  0:00:01  0:00:01 --:--:-- 49.0M


In [2]:
!tar -xf aclImdb_v1.tar.gz

In [3]:
!rm -r aclImdb/train/unsup

In [4]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [5]:
import os ,pathlib, shutil,random

In [7]:
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir/"val"
train_dir = base_dir/"train"
for category in ("neg","pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2*len(files))
  val_files = files[:num_val_samples]
  for fname in val_files:
    shutil.move(train_dir/category/fname,
                val_dir/category/fname)

In [8]:
from tensorflow import keras

In [9]:
train_ds = keras.utils.text_dataset_from_directory('aclImdb/train',
                                                   batch_size=32)
val_ds = keras.utils.text_dataset_from_directory('aclImdb/val',
                                                   batch_size=32)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test',
                                                   batch_size=32)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [43]:
for x,y in train_ds:
  print(x.shape, y.shape)
  print("x[0]:",x[0])
  print("y[0]:",y[0])
  break

(32,) (32,)
x[0]: tf.Tensor(b"I had never seen such an incredible acting job in a motion picture as I did when I saw Daniel Day-Lewis play Christy Brown in My Left Foot. In fact off the scene his role wasn't even over. He played the role of Christy Brown or at least disabled like him all through the filming of the movie and needed surgery because of the damage his superior acting had done to his back. To me that is remarkable and through all the pain he put up with to act that role I believe it is quite true to say he put on the most Oscar worthy performance in history. He was so masterful in this tough a part that I believe no one could have done it better or with more of an impact than him. Although I cannot say it is the greatest movie of all time I can say that how he played this impossible a role and then kept on acting it until it wasn't even acting anymore is without a doubt the greatest feet I will ever seen an actor do. Probably a man too for that matter.", shape=(), dtype=str

In [64]:
tv = layers.TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x,y: x)
tv.adapt(text_only_train_ds) #vacabulary 형성

In [67]:
for batch in text_only_train_ds:
  for text in batch:
    print(text)
    print(tv(text))
    print(tv(text).shape)
    break
  break

tf.Tensor(b"This is Not a Love Song.<br /><br />My one word summary of this film would be `Excellent'.<br /><br />It probably won't appeal to the mass movie watching public \xc2\x96 it's a<br /><br />film that forces you to participate. You observe, think, and question.<br /><br />Comparisons could be made with Deliverance (Topic/Theme) and<br /><br />perhaps with The Blair Witch Project for overall filming style.<br /><br />However this film stands unique against both.<br /><br />The cinematography effects (solarisation, freeze frame, blur etc)<br /><br />have been seen before but they are used most effectively in this<br /><br />film to underpin the natural tension of the story.<br /><br />Acting is raw, menacing and utterly believable.<br /><br />The real theme of the film is about friendship; the title really gives<br /><br />the game away. It's probably not the kind of friendship that most of<br /><br />us have experienced or indeed would want to.<br /><br />It is a love song.", s

In [68]:
binary_1gram_train_ds = train_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
binary_1gram_val_ds = val_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
binary_1gram_test_ds = test_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)

In [69]:
for x,y in binary_1gram_train_ds:
  print(x.shape, y.shape)
  print("x[0]:",x[0])
  print("y[0]:",y[0])
  break

(32, 20000) (32,)
x[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
y[0]: tf.Tensor(1, shape=(), dtype=int32)


In [70]:
def get_model(max_tokens=20000,hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim,activation='relu')(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1,activation='sigmoid')(x)
  model = keras.Model(inputs,outputs)
  model.compile(loss=keras.losses.binary_crossentropy,
                metrics=['accuracy'])
  return model

In [71]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [73]:
mcp = keras.callbacks.ModelCheckpoint('binary_1gram.keras',
                                      save_best_only=True)
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('binary_1gram.keras')
model.evaluate(binary_1gram_test_ds)

Epoch 1/10
625/625 [==============================] - 8s 12ms/step - loss: 0.4162 - accuracy: 0.8257 - val_loss: 0.3005 - val_accuracy: 0.8812
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2804 - accuracy: 0.8982 - val_loss: 0.2989 - val_accuracy: 0.8858
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2453 - accuracy: 0.9146 - val_loss: 0.3045 - val_accuracy: 0.8850
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2234 - accuracy: 0.9240 - val_loss: 0.3312 - val_accuracy: 0.8788
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2204 - accuracy: 0.9276 - val_loss: 0.3292 - val_accuracy: 0.8810
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2103 - accuracy: 0.9305 - val_loss: 0.3550 - val_accuracy: 0.8788
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2056 - accuracy: 0.9340 - val_loss: 0.3581 - val_accuracy: 0.8782
Epoch

[0.2933864891529083, 0.8814399838447571]

In [74]:
tv = layers.TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x,y: x)
tv.adapt(text_only_train_ds) #vacabulary 형성

In [75]:
binary_2gram_train_ds = train_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
binary_2gram_val_ds = val_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
binary_2gram_test_ds = test_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)

In [76]:
model = get_model()
mcp = keras.callbacks.ModelCheckpoint('binary_2gram.keras',
                                      save_best_only=True)
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('binary_2gram.keras')
model.evaluate(binary_2gram_test_ds)

Epoch 1/10
625/625 [==============================] - 7s 10ms/step - loss: 0.3709 - accuracy: 0.8441 - val_loss: 0.2871 - val_accuracy: 0.8858
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2334 - accuracy: 0.9177 - val_loss: 0.2782 - val_accuracy: 0.9008
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2046 - accuracy: 0.9333 - val_loss: 0.2997 - val_accuracy: 0.9002
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1811 - accuracy: 0.9430 - val_loss: 0.3165 - val_accuracy: 0.9018
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1802 - accuracy: 0.9480 - val_loss: 0.3275 - val_accuracy: 0.8976
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1700 - accuracy: 0.9522 - val_loss: 0.3641 - val_accuracy: 0.8982
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1586 - accuracy: 0.9528 - val_loss: 0.3736 - val_accuracy: 0.9002
Epoch

[0.27281391620635986, 0.900160014629364]

In [77]:
tv = layers.TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count",
)
text_only_train_ds = train_ds.map(lambda x,y: x)
tv.adapt(text_only_train_ds) #vacabulary 형성

In [78]:
count_2gram_train_ds = train_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
count_2gram_val_ds = val_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
count_2gram_test_ds = test_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)

In [79]:
model = get_model()
mcp = keras.callbacks.ModelCheckpoint('count_2gram.keras',
                                      save_best_only=True)
model.fit(count_2gram_train_ds.cache(),
          validation_data=count_2gram_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('count_2gram.keras')
model.evaluate(count_2gram_test_ds)

Epoch 1/10
625/625 [==============================] - 6s 9ms/step - loss: 0.4508 - accuracy: 0.7955 - val_loss: 0.3099 - val_accuracy: 0.8768
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3036 - accuracy: 0.8737 - val_loss: 0.2786 - val_accuracy: 0.8982
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2662 - accuracy: 0.8942 - val_loss: 0.2891 - val_accuracy: 0.8944
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2427 - accuracy: 0.9068 - val_loss: 0.2943 - val_accuracy: 0.8982
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2300 - accuracy: 0.9122 - val_loss: 0.3247 - val_accuracy: 0.8884
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2248 - accuracy: 0.9137 - val_loss: 0.3190 - val_accuracy: 0.8906
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2135 - accuracy: 0.9180 - val_loss: 0.3456 - val_accuracy: 0.8888
Epoch 

[0.2678576707839966, 0.8980399966239929]

In [80]:
tv = layers.TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)
text_only_train_ds = train_ds.map(lambda x,y: x)
tv.adapt(text_only_train_ds) #vacabulary 형성

In [81]:
tfidf_2gram_train_ds = train_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
tfidf_2gram_val_ds = val_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
tfidf_2gram_test_ds = test_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)

In [82]:
model = get_model()
mcp = keras.callbacks.ModelCheckpoint('tfidf_2gram.keras',
                                      save_best_only=True)
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('tfidf_2gram.keras')
model.evaluate(tfidf_2gram_test_ds)

Epoch 1/10
625/625 [==============================] - 6s 9ms/step - loss: 0.4798 - accuracy: 0.7793 - val_loss: 0.4122 - val_accuracy: 0.8490
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3191 - accuracy: 0.8646 - val_loss: 0.2981 - val_accuracy: 0.8926
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2816 - accuracy: 0.8850 - val_loss: 0.3902 - val_accuracy: 0.8502
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2559 - accuracy: 0.8984 - val_loss: 0.3604 - val_accuracy: 0.8892
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2437 - accuracy: 0.9035 - val_loss: 0.3361 - val_accuracy: 0.8914
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2163 - accuracy: 0.9096 - val_loss: 0.3575 - val_accuracy: 0.8906
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2098 - accuracy: 0.9109 - val_loss: 0.3606 - val_accuracy: 0.8870
Epoch 

[0.2958030700683594, 0.8955600261688232]

## 시퀀스

In [12]:
from tensorflow import keras
from keras import layers

In [13]:
max_length = 600
max_tokens= 20000

In [14]:
tv = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_only_train_ds = train_ds.map(lambda x,y: x)
tv.adapt(text_only_train_ds) #vacabulary 형성

In [15]:
int_train_ds = train_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
int_val_ds = val_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
int_test_ds = test_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)

In [10]:
import tensorflow as tf

In [88]:
inputs = keras.Input(shape=(None,),dtype='int64')
embedded = tf.one_hot(inputs,depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model = keras.Model(inputs,outputs)
model.compile(loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [ ]:
mcp = keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm.keras',
                                      save_best_only=True)
model.fit(int_train_ds.cache(),
          validation_data=int_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('one_hot_bidir_lstm.keras')
model.evaluate(int_test_ds)

Epoch 1/10


In [16]:
inputs = keras.Input(shape=(None,),dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens,output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model = keras.Model(inputs,outputs)
model.compile(loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [17]:
mcp = keras.callbacks.ModelCheckpoint('embedding_bidir_lstm.keras',
                                      save_best_only=True)
model.fit(int_train_ds.cache(),
          validation_data=int_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('embedding_bidir_lstm.keras')
model.evaluate(int_test_ds)

Epoch 1/10
625/625 [==============================] - 101s 147ms/step - loss: 0.5265 - accuracy: 0.7377 - val_loss: 0.4455 - val_accuracy: 0.8288
Epoch 2/10
625/625 [==============================] - 22s 35ms/step - loss: 0.3584 - accuracy: 0.8644 - val_loss: 0.4313 - val_accuracy: 0.8460
Epoch 3/10
625/625 [==============================] - 22s 35ms/step - loss: 0.3085 - accuracy: 0.8873 - val_loss: 0.3211 - val_accuracy: 0.8692
Epoch 4/10
625/625 [==============================] - 22s 35ms/step - loss: 0.2371 - accuracy: 0.9197 - val_loss: 0.3609 - val_accuracy: 0.8730
Epoch 5/10
625/625 [==============================] - 22s 35ms/step - loss: 0.2113 - accuracy: 0.9276 - val_loss: 0.3388 - val_accuracy: 0.8672
Epoch 6/10
625/625 [==============================] - 22s 35ms/step - loss: 0.1829 - accuracy: 0.9413 - val_loss: 0.3670 - val_accuracy: 0.8742
Epoch 7/10
625/625 [==============================] - 23s 36ms/step - loss: 0.1603 - accuracy: 0.9484 - val_loss: 0.4123 - val_accurac

[0.34273719787597656, 0.8553599715232849]